In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as io
import pandas as pd
from pandas import DataFrame as dframe
import cvxpy as cv

In [2]:
#対象被験者の特徴ベクトル(トレーニングデータ＆テストデータ)をMATLAB(自身で作成)から入手
feat_C1 = np.loadtxt("feat_C1.txt",delimiter=",")
feat_C2 = np.loadtxt("feat_C2.txt",delimiter=",")
feat_Test_C1 = np.loadtxt("feat_Test_C1.txt",delimiter=",")
feat_Test_C2 = np.loadtxt("feat_Test_C2.txt",delimiter=",")

In [3]:
#非対象被験者の特徴ベクトルをMATLABから入手
feat_C1_K1 = np.loadtxt("feat_C1_K1.txt",delimiter=",")
feat_C2_K1 = np.loadtxt("feat_C2_K1.txt",delimiter=",")

In [4]:
#辞書行列を作る関数
def Dic(feat_l_tr,feat_r_tr): #入力の配列は2次元配列である必要がある
    feat_tr = np.concatenate([feat_l_tr, feat_r_tr],0)
    return feat_tr

In [5]:
#対象被験者の各クラスのトレーニングデータから取得した特徴ベクトルを結合
feat_tr = Dic(feat_C1,feat_C2)
#非対象被験者の特徴ベクトルも同様の処理を行う
feat_tr_K1 = Dic(feat_C1_K1,feat_C2_K1)
#それぞれの特徴ベクトルをつなげる
#この時、2M×Nの行列にするために転置させる(2Mは特徴ベクトルの次元、Nはスパース表現を推定するために使用した特徴ベクトルの総数)
feat_tr_dic = Dic(feat_tr,feat_tr_K1).T
#feat_tr_dicはすべての特徴ベクトルをつなげたもの
feat_l_te = feat_Test_C1;
feat_r_te = feat_Test_C2;
#テストデータの特徴ベクトルも2Mが行になるように転置させる
feat_l_te = feat_l_te.T
feat_r_te = feat_r_te.T
print(feat_l_te.shape,feat_tr_dic.shape,feat_tr.shape,feat_tr_K1.shape)

(4, 12) (4, 200) (120, 4) (80, 4)


In [1]:
#各クラスの特徴ベクトルの配列の列数を格納した配列を引数にすればもう少し拡張性が上がる
def SGRM(feat_C1,feat_C2,feat_tr,feat_tr_K1,feat_tr_dic,feat_C1_K1,feat_C2_K1,feat_l_te):
    #今回ハイパーパラメータは0.01に設定している
    λ_1 = 0.01
    λ_2 = 0.01
    answer = np.zeros(feat_l_te.shape[1])
    #スパース性を格納する行列
    SGRM_Cla = np.zeros((feat_tr_dic.shape[1], feat_l_te.shape[1]))
    #leftクラスのみのスパース性を格納するための行列
    SGRM_l_Cla = np.zeros((feat_tr_dic.shape[1], feat_l_te.shape[1]))
    #rightクラスのみのスパース性を格納するための行列
    SGRM_r_Cla = np.zeros((feat_tr_dic.shape[1], feat_l_te.shape[1]))
    for i in range(feat_l_te.shape[1]):
        u = cv.Variable((feat_tr_dic.shape[1],1)) #目的関数の次元
        #SGRMの最適化問題を設定している
        objective = cv.Minimize(0.5*(cv.norm(feat_tr_dic@u-feat_l_te[:,i,np.newaxis],2)**2)
                                +λ_1*cv.norm(u,1)
                                +λ_2*(cv.norm(u[0:feat_tr.shape[0],0],2)
                                      +cv.norm(u[feat_tr.shape[0]:feat_tr.shape[0]+feat_tr_K1.shape[0],0],2)))
        prob = cv.Problem(objective)
        result = prob.solve()
        #classVariableから値を取るためには.valueを付け、次元も削減する
        SGRM_Cla[:,i] = np.squeeze(u.value)
        #各クラスに関連する表現ベクトルの値を抜き出す
        #lは仕切りの値
        l = 0
        SGRM_l_Cla[0:feat_C1.shape[0],i] = SGRM_Cla[0:feat_C1.shape[0],i]     
        l = l + feat_C1.shape[0]
        SGRM_r_Cla[l:l+feat_C2.shape[0],i] = SGRM_Cla[l:l+feat_C2.shape[0],i]
        l = l + feat_C2.shape[0]
        SGRM_l_Cla[l:l+feat_C1_K1.shape[0],i] = SGRM_Cla[l:l+feat_C1_K1.shape[0],i]
        l = l + feat_C1_K1.shape[0]
        SGRM_r_Cla[l:l+feat_C2_K1.shape[0],i] = SGRM_Cla[l:l+feat_C2_K1.shape[0],i]
    #パラメータを使って擬似的に各クラスのテスト信号の特徴ベクトルを作成
    dout_feat_l_te = np.zeros((feat_l_te.shape[0], feat_l_te.shape[1]))
    dout_feat_r_te = np.zeros((feat_l_te.shape[0], feat_l_te.shape[1])) 
    for i in range(feat_l_te.shape[1]):
        dout_feat_l_te[:,i] = feat_tr_dic@SGRM_l_Cla[:,i]
    for i in range(feat_r_te.shape[1]):
        dout_feat_r_te[:,i] = feat_tr_dic@SGRM_r_Cla[:,i]
    #作成した擬似特徴ベクトルとの残差を求める
    R_l =  np.zeros((feat_l_te.shape[0], feat_l_te.shape[1])) #残差を格納するための配列
    R_r =  np.zeros((feat_l_te.shape[0], feat_l_te.shape[1]))  
    for i in range(feat_l_te.shape[1]):
        R_l[:,i] = feat_l_te[:,i]-dout_feat_l_te[:,i]
        R_r[:,i] = feat_l_te[:,i]-dout_feat_r_te[:,i]
        #残差を比べてクラスを割り当てる
        if np.linalg.norm(R_l[:,i], ord=2) < np.linalg.norm(R_r[:,i], ord=2):
            answer[i] = 1
        elif np.linalg.norm(R_l[:,i], ord=2) > np.linalg.norm(R_r[:,i], ord=2):
            answer[i] = 2
    return answer

In [ ]:
#正答率を割り出す
answer = SGRM(feat_C1,feat_C2,feat_tr,feat_tr_K1,feat_tr_dic,feat_C1_K1,feat_C2_K1,feat_l_te)
x = 0
for i in range(answer.shape[0]):
    if answer[i] == 1:
        x = x + 1
print((x/answer.shape[0])*100)